In [269]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [270]:
df = pd.read_csv("/content/testset.csv")
df.head()

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.0,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.2,0.0,North,NaN,NaN,0.0


In [271]:
df['datetime_utc'] = pd.to_datetime(df['datetime_utc'], format='%Y%m%d-%H:%M')


In [272]:
df.columns

Index(['datetime_utc', ' _conds', ' _dewptm', ' _fog', ' _hail',
       ' _heatindexm', ' _hum', ' _precipm', ' _pressurem', ' _rain', ' _snow',
       ' _tempm', ' _thunder', ' _tornado', ' _vism', ' _wdird', ' _wdire',
       ' _wgustm', ' _windchillm', ' _wspdm'],
      dtype='object')

In [273]:
df['hour'] = df['datetime_utc'].dt.hour
df['day'] = df['datetime_utc'].dt.day
df['month'] = df['datetime_utc'].dt.month
df['year'] = df['datetime_utc'].dt.year


In [274]:
features = [' _dewptm', ' _hum', ' _vism', ' _tempm', ' _pressurem', 'hour', 'day', 'month', 'year']
df = df[features + [' _fog']].dropna()

In [275]:
df = df[features + [' _fog']].dropna()

In [276]:
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [277]:
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [278]:
X = df[features].values
y = df[' _fog'].values.reshape(-1, 1)

In [279]:
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [280]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [281]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [282]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.9478 - loss: 0.2058 - val_accuracy: 0.9602 - val_loss: 0.0886
Epoch 2/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9595 - loss: 0.0907 - val_accuracy: 0.9613 - val_loss: 0.0876
Epoch 3/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9628 - loss: 0.0845 - val_accuracy: 0.9583 - val_loss: 0.0897
Epoch 4/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9623 - loss: 0.0867 - val_accuracy: 0.9620 - val_loss: 0.0866
Epoch 5/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9617 - loss: 0.0866 - val_accuracy: 0.9613 - val_loss: 0.0876
Epoch 6/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9613 - loss: 0.0885 - val_accuracy: 0.9598 - val_loss: 0.0882
Epoch 7/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9630 - loss: 0.0842 - val_accuracy: 0.9619 - val_loss: 0.0869
Epoch 8/20
1846/1846 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9634 - loss: 0.082

In [283]:
y_pred = model.predict(X_test)


462/462 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [284]:
df['fog_density'] = model.predict(X).flatten()


2308/2308 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [289]:
# Find the timestamp with the highest fog density
max_fog_index = df['fog_density'].idxmax()
max_fog_time = df.loc[max_fog_index, 'hour']
max_fog_density = df.loc[max_fog_index, 'fog_density']

print(f"The fog is densest at {max_fog_time} with a fog density of {max_fog_density:.4f}")


The fog is densest at 1.0 with a fog density of 0.9417


In [290]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert predictions to binary (0 or 1)
y_pred_binary = (model.predict(X_test) > 0.5).astype(int)

# Calculate Classification Metrics
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


462/462 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9640
Precision: 0.6592
Recall: 0.5938
F1 Score: 0.6248
